# 10. Diseño de Algoritmos 🚀

- *Autor*: [Dr. Mario Abarca](https://www.knkillname.org/)
- *Objetivos*: Comprender y aplicar estrategias de diseño de algoritmos: fuerza bruta, vuelta atrás, voraces y programación dinámica.

En este cuaderno aprenderemos "recetas para hacer recetas". ¿Cómo encuentran los programas la ruta más corta 🗺️ o juegan ajedrez ♟️? La respuesta está en el diseño de algoritmos.

## 10.3 Estrategia de Fuerza Bruta: ¡Probando Todas las Opciones! 🕵️‍♀️

La **fuerza bruta** es la estrategia más directa: probar *todas* las posibilidades hasta encontrar la correcta. Es como abrir un candado probando 000, 001, 002... hasta que abra.

Es fácil de implementar, pero si el "espacio de búsqueda" es grande, puede ser extremadamente lenta ⏳.

### El Dilema de la Mochila (0/1 Knapsack) 🎒

Tienes una mochila con capacidad $W$ y varios objetos, cada uno con un peso $w$ y un valor $v$. Quieres maximizar el valor total sin exceder el peso.

**Ejemplo**: Examen de 60 min. Preguntas con diferentes puntos y tiempos.
¿Qué preguntas contestar para sacar la máxima nota?

La fuerza bruta probaría **todas las combinaciones posibles** de preguntas y se quedaría con la mejor que quepa en 60 min.

### Ejercicios 🧠

1.  **Variantes de la Mochila**:
    *   Investiga: ¿Qué es el "problema de la mochila fraccional"? ¿En qué se diferencia del 0/1?
    *   Intenta pensar cómo resolverías la fraccional con una idea simple (¡quizás no necesites fuerza bruta!).

2.  **Tu Propio Problema**:
    *   Piensa en una situación diaria parecida (ej: instalar juegos en el celular con poco espacio). Define objetos, pesos y valores.

### Discusión con tu IA 🤖

1.  **Mochila Fraccional vs 0/1**:
    *   Pregúntale a tu asistente: ¿Cuándo usarías la mochila 0/1 y cuándo la fraccional? ¿Cuál es computacionalmente más difícil?

2.  **Fuerza Bruta vs. Inteligencia**:
    *   Reflexiona: ¿Por qué no siempre usamos fuerza bruta si es tan fácil? ¿Cuándo es útil a pesar de su lentitud?

### Implementación de Fuerza Bruta 🎒💪

Para $n$ objetos, hay $2^n$ combinaciones posibles (cada objeto puede estar dentro o fuera).
El algoritmo:
1.  Genera todas las combinaciones (conjunto potencia).
2.  Filtra las que exceden el peso.
3.  Elige la de mayor valor.

¡Veamos el código! 🐍

In [ ]:
from itertools import chain, combinations


def conjunto_potencia(iterable):
    elementos = list(iterable)
    yield from chain.from_iterable(
        combinations(elementos, r) for r in range(len(elementos) + 1)
    )

In [ ]:
def mochila_fuerza_bruta(capacidad, pesos, valores):
    cantidad_elementos = len(pesos)
    if cantidad_elementos != len(valores):
        raise ValueError("Las listas de pesos y valores deben tener la misma longitud.")

    mejor_valor, mejor_seleccion = 0, []

    # Generar todos los subconjuntos posibles usando conjunto_potencia
    for subconjunto in conjunto_potencia(range(cantidad_elementos)):
        peso_actual = sum(pesos[i] for i in subconjunto)
        valor_actual = sum(valores[i] for i in subconjunto)

        # Verificar si el subconjunto actual es válido y si mejora el mejor valor encontrado
        if peso_actual <= capacidad and valor_actual > mejor_valor:
            mejor_valor = valor_actual
            mejor_seleccion = subconjunto

    return mejor_valor, mejor_seleccion

Ejemplo de uso:

In [ ]:
capacidad = 10
pesos = [2, 3, 4, 5]
valores = [3, 7, 2, 9]
mejor_valor, mejor_seleccion = mochila_fuerza_bruta(capacidad, pesos, valores)

In [ ]:
mejor_valor

In [ ]:
mejor_seleccion

### Análisis de Complejidad 📈

*   **Temporal**: $O(n \cdot 2^n)$. Generar $2^n$ subconjuntos y sumarlos. Crece explosivamente. Para $n=40$, ¡es imposible!
*   **Espacial**: $O(n)$. Solo guardamos la mejor selección.

**Conclusión**: La fuerza bruta solo sirve para problemas pequeños.

### Generalización
Esta estrategia (Búsqueda Exhaustiva) sirve para cualquier problema donde puedas listar soluciones:
1.  Generar candidatas.
2.  Validar reglas.
3.  Evaluar calidad.
4.  Quedarse con la mejor.

## 10.4 Vuelta Atrás (Backtracking) 🚶‍♀️🔄

Si estás en un laberinto y llegas a un callejón sin salida, regresas a la última bifurcación y pruebas otro camino. ¡Eso es **Backtracking**!

A diferencia de la fuerza bruta, el backtracking es astuto: **poda** las ramas que no llevan a solución. Si una opción parcial ya rompe las reglas, la descarta inmediatamente.

### Ejemplo: N Reinas 👑
Colocar $N$ reinas en un tablero $N \times N$ sin que se ataquen.

**Estrategia**:
1.  Colocar reina en columna actual.
2.  ¿Es seguro?
    *   **Sí**: Avanzar a siguiente columna.
    *   **No**: Probar siguiente fila.
3.  Si no hay filas seguras, **retroceder** (backtrack) a la columna anterior y mover esa reina.

Representamos el tablero como una lista donde `tablero[columna] = fila`.

In [ ]:
def es_valida(tablero, fila, columna):
    for col_anterior in range(columna):
        fila_anterior = tablero[col_anterior]
        # Verificar si están en la misma fila o en la misma diagonal
        if fila_anterior == fila or abs(fila_anterior - fila) == abs(
            col_anterior - columna
        ):
            return False
    return True

In [ ]:
# ¿Podemos colocar una reina en la fila 2, columna 3 de un tablero con las reinas en las
# posiciones (0, 0), (1, 2) y (2, 4)?
tablero = [0, 2, 4, -1, -1, -1, -1, -1]
es_valida(tablero, 2, 3)

In [ ]:
def resolver_reinas(tablero, j, n):
    # Sup. que ya colocamos reinas en las primeras columnas.

    if j == n:  # ¿Hemos colocado todas las reinas?
        print(tablero)  # Imprimir solución encontrada
        return

    for i in range(n):  # Probar cada fila de la columna j
        if es_valida(tablero, i, j):  # ¿Podemos colocar una reina en la fila i?
            tablero[j] = i  # Colocar la reina en la fila i, columna j
            resolver_reinas(tablero, j + 1, n)  # Explorar la siguiente columna


def problema_reinas(n):
    tablero = [-1] * n  # Inicializar el tablero con valores inválidos
    resolver_reinas(tablero, 0, n)

In [ ]:
# Resolver el problema de las 8 reinas
problema_reinas(8)

### Análisis de Complejidad 🧐

El peor caso es $O(n!)$, que es incluso peor que exponencial. Sin embargo, gracias a la **poda**, en la práctica es mucho más rápido que la fuerza bruta.

Se usa en: Sudokus 🔢, Laberintos 🗺️, Horarios 📅.

### Ejercicios 🧠

1.  **Reinas con un Giro 👑🔄**:
   - ¿Qué pasaría si el tablero fuera como un donut (toroidal, donde el lado derecho se conecta con el izquierdo y el de arriba con el de abajo)? ¿O si algunas casillas estuvieran prohibidas 🚫 desde el inicio? Investiga estas variantes del problema de las N Reinas.
   - Elige una variante ¡y programa una solución usando vuelta atrás!

2.  **Fuerza Bruta vs. Vuelta Atrás 🥊**:
   - Programa la solución al problema de las N Reinas de dos formas:
      1.  Probando *todo* sin piedad (fuerza bruta).
      2.  Usando la técnica inteligente de 'vuelta atrás'.
   - Mide cuánto tarda cada uno para diferentes números de reinas (por ejemplo, para $n=4, n=5, \ldots, n=8$).

3.  **Más Desafíos de Vuelta Atrás 🧩**:
   - Hay muchos otros problemas famosos que se resuelven muy bien con esta técnica:
      - Resolver un Sudoku.
      - El problema del recorrido del caballo en ajedrez (¿puede un caballo visitar todas las casillas de un tablero sin pasar dos veces por la misma? 🐴).
      - El problema de partición de conjuntos (dado un conjunto de números, ¿puedes dividirlo en dos subconjuntos cuyas sumas sean iguales?).
   - Elige uno que te interese, ¡intenta programar una solución!

4.  **¡A Dibujar el Laberinto! 🎨🗺️**:
   - ¿Te animas a crear un programa que muestre visualmente cómo la 'vuelta atrás' explora las soluciones para las N Reinas? Podrías dibujar el tablero y cómo se van poniendo y quitando las reinas, ¡y marcar los caminos que descarta!

### Discusión con tu IA 🤖

1.  **Variantes del Problema**:
   - Pregunta a tu IA amiga: ¿Los cambios como el tablero toroidal o casillas prohibidas hacen el problema más fácil o más difícil de resolver computacionalmente? ¿Por qué?

2.  **Comparación de Rendimiento**:
   - Después de medir los tiempos en el ejercicio 2, discute con tu IA: ¿Cuándo brilla ✨ la vuelta atrás y por qué crees que es así? ¿Qué pasa con la complejidad asintótica en el caso promedio vs el peor caso?

3.  **¿Hay Otras Formas? 🤔**:
   - Investiga si otros 'trucos' o estrategias para diseñar algoritmos (como los algoritmos voraces que vimos, o la programación dinámica que veremos pronto) podrían servir para problemas parecidos al de las N Reinas.
   - Si encuentras alguna estrategia alternativa, ¿cómo crees que se compararía con la 'vuelta atrás' en términos de encontrar la solución o de rapidez? ¡Discútelo!

## 10.5 Algoritmos Voraces (Greedy) 😋

Un algoritmo voraz toma la mejor decisión **local** en cada paso, esperando llegar a la solución **global**.
*   *Ventaja*: Rápidos y sencillos.
*   *Riesgo*: No siempre encuentran la solución óptima (ej: problema de la mochila).

### Ejemplo: Árbol de Expansión Mínima (MST) 🌳🔌
Conectar todos los edificios de la UAEM con el menor costo de cable posible.

**Algoritmo de Prim**:
1.  Empieza en un edificio cualquiera.
2.  De todos los cables que salen de tu red actual hacia fuera, elige el **más barato**.
3.  Añádelo y repite hasta conectar todo.

¡Siempre elige lo más barato disponible *ahora*! Para el MST, esto garantiza la solución óptima.

In [ ]:
# Grafo: Diccionario de diccionarios {origen: {destino: costo}}
grafo_ejemplo = {
    "A": {"B": 2, "C": 3},
    "B": {"A": 2, "C": 1},
    "C": {"A": 3, "B": 1, "D": 4},
    "D": {"C": 4},
}
print(f"Costo de B a C: {grafo_ejemplo['B']['C']}")

### Implementación de Prim 🐍

Usaremos una **Cola de Prioridad** (`heapq`) para obtener siempre el cable más barato eficientemente.

1.  `heapq` guarda `(costo, origen, destino)`.
2.  Sacamos el menor costo.
3.  Si el destino no está conectado, lo añadimos y metemos sus conexiones al heap.

In [ ]:
import heapq


def algoritmo_prim(grafo):
    if not grafo:
        return [], 0  # Grafo vacío, sin MST.

    nodo_inicio = list(grafo)[0]  # Elegir nodo de inicio.
    nodos_en_mst = {nodo_inicio}  # Nodos en el MST.
    aristas_mst = []  # Aristas del MST.
    costo_total_mst = 0  # Costo total del MST.
    # Cola de prioridad para aristas candidatas (costo, origen, destino):
    aristas_candidatas_cp = []

    # Inicializar cola con aristas del nodo de inicio.
    for vecino, costo in grafo[nodo_inicio].items():
        heapq.heappush(aristas_candidatas_cp, (costo, nodo_inicio, vecino))

    # Bucle hasta que todos los nodos estén en el MST o no haya más aristas.
    while len(nodos_en_mst) < len(grafo) and aristas_candidatas_cp:
        # Extraer arista de menor costo:
        costo_arista, origen, destino = heapq.heappop(aristas_candidatas_cp)

        if destino in nodos_en_mst:
            continue  # Si el destino ya está en el MST, ignorar (evitar ciclo).

        # Añadir nodo y arista al MST.
        nodos_en_mst.add(destino)
        aristas_mst.append((origen, destino))
        costo_total_mst += costo_arista

        # Añadir nuevas aristas candidatas desde el nodo recién agregado.
        for vecino_de_destino, costo_hacia_vecino in grafo[destino].items():

            if vecino_de_destino not in nodos_en_mst:  # Considerar solo vecinos fuera.
                # Añadir nueva arista candidata a la cola.
                heapq.heappush(
                    aristas_candidatas_cp,
                    (costo_hacia_vecino, destino, vecino_de_destino),
                )

    return aristas_mst, costo_total_mst

In [ ]:
# Ejemplo de grafo representando edificios (letras) y costos de conexión (enteros)
ejemplo = {
    "Rectoria": {"Fac_Ciencias": 5, "Deportes": 8, "Ingenieria": 12},
    "Fac_Ciencias": {"Rectoria": 5, "Deportes": 9, "Posgrado_A": 4},
    "Deportes": {"Rectoria": 8, "Fac_Ciencias": 9, "Posgrado_A": 7, "Biblioteca": 6},
    "Ingenieria": {"Rectoria": 12, "Biblioteca": 3},
    "Posgrado_A": {"Fac_Ciencias": 4, "Deportes": 7, "Biblioteca": 2},
    "Biblioteca": {"Deportes": 6, "Ingenieria": 3, "Posgrado_A": 2},
}

# Ejecutar el algoritmo
aristas_minimas, costo_total = algoritmo_prim(ejemplo)

for origen, destino in aristas_minimas:
    print(f"Conectar {origen} con {destino}")
print(f"Costo total del MST: {costo_total}")

### Complejidad de Prim ⏱️

Gracias al `heapq`, la complejidad es $O(|E| \log |V|)$.
*   $|E|$ aristas (cables).
*   $|V|$ vértices (edificios).
Es muy eficiente incluso para redes grandes.

### Ejercicios 🧠

1.  **El MST con un Toque Diferente 🔄**:
    - ¿Qué tal si le damos una vuelta de tuerca al problema del Árbol de Expansión Mínima? Investiga algunas variantes:
      - **Árbol de Expansión Máxima**: Imagina que en vez de buscar el costo *mínimo*, ¡queremos el *máximo*! 🤑
      - **MST con Restricciones**: ¿Y si algunas conexiones (aristas) estuvieran prohibidas 🚫 o tuviéramos que incluir sí o sí alguna conexión específica?
    - Elige una de estas nuevas versiones y ¡anímate a programar una solución voraz! 💻

2.  **Prim vs. Kruskal: Duelo de Titanes 🥊**:
    - ¡Hay más de un camino para encontrar el MST! Ya conoces el algoritmo de Prim. Investiga y luego implementa el **algoritmo de Kruskal**.
    - Ponlos a competir ⏱️: ¿cuál es más rápido? Prueba con diferentes tipos de mapas (grafos): unos con muchos caminos posibles (densos) y otros con poquitos (dispersos).

3.  **¡Dibuja cómo Piensa Prim! 🎨**:
    - ¡Hagamos que el algoritmo de Prim cobre vida! Crea un programa que muestre visualmente cómo va eligiendo los cables (aristas) paso a paso para construir el MST en un grafo de ejemplo.

4.  **Más Allá del MST: Otros Problemas Voraces 🌍**:
    - ¡El mundo voraz es grande! Investiga otros problemas famosos que se resuelven con esta técnica. Algunos ejemplos son:
      - **El problema del cambio de monedas** 💰: ¿Cómo darías el cambio usando la menor cantidad de monedas posible?
      - **La codificación de Huffman**: Una técnica usada para comprimir archivos 📁 de forma eficiente.
      - **El problema de selección de actividades**: Si tienes muchas actividades con horarios de inicio y fin, ¿cómo eliges el máximo número de actividades sin que se solapen? 🗓️
    - Elige uno que te interese, ¡prográmalo! 🚀

### Discusión con tu IA 🤖

1.  **Ajustes al Algoritmo**:
    - Charla con tu IA amiga: ¿Sigue sirviendo el algoritmo de Prim tal cual para el Árbol de Expansión Máxima, o necesita ajustes? ¿Qué cambios harías?

2.  **Comparativa Prim vs Kruskal**:
    - Reflexiona con tu asistente: ¿Hay situaciones donde uno es claramente el campeón 🏆 y otras donde el otro brilla más? ¿A qué crees que se debe? (Pista: piensa en grafos densos vs dispersos).

3.  **Cuando Ser Voraz Falla 😬**:
    - A veces, tomar la mejor decisión *ahora* no siempre lleva a la mejor solución *global*.
    - Pídele a tu IA que te muestre un contraejemplo donde un algoritmo voraz falle (por ejemplo, en el problema del cambio de monedas con denominaciones extrañas, o el problema del viajante de comercio).
    - Discute: ¿Hay señales o pistas para saber cuándo una estrategia voraz podría no ser la mejor idea?

### 10.5 Montículos (Heaps) 🏗️

Este tema es tan importante y tiene tantas aplicaciones prácticas que hemos decidido moverlo a su propia práctica dedicada. En ella, aprenderás a gestionar una **Sala de Urgencias Zombie** 🧟‍♀️ usando colas de prioridad.

👉 **[Ir a la Práctica 7: Montículos y Colas de Prioridad](../prácticas/7.Monticulos.md)**

## 10.6 Programación Dinámica 🧠✨

Ya vimos cómo los algoritmos voraces toman la "mejor" decisión en el momento. Ahora, vamos a conocer una técnica súper poderosa llamada **Programación Dinámica** (¡o PD para los amigos!).

Imagina que tienes un problema enorme y complicado, ¡como armar un rompecabezas gigante 🧩! La PD nos ayuda a resolver estos monstruos de problemas de una forma muy astuta. En lugar de probarlo todo a lo loco, la PD busca patrones y "recuerda" soluciones a pedacitos más pequeños del problema.

**¿Cuándo brilla la Programación Dinámica?** 🌟

La PD es la estrella cuando un problema tiene dos características especiales:

1.  **Subproblemas que se Repiten (Subproblemas Superpuestos)**:
   Piensa en el rompecabezas gigante. Es muy probable que, mientras lo armas, te encuentres con que necesitas resolver el mismo "mini-rompecabezas" (una esquina específica, un grupito de piezas) ¡varias veces! 😱 La PD dice: "¡Oye, esto ya lo resolví antes! No necesito hacerlo de nuevo." Guarda la solución de ese mini-rompecabezas y la reutiliza. ¡Qué listo!

2.  **La Mejor Solución se Construye con las Mejores "Mini-Soluciones" (Estructura Óptima de Subestructura)**:
   Esto suena complicado, ¡pero es simple! Significa que si encuentras la mejor manera de resolver cada uno de esos "mini-rompecabezas", entonces, al juntarlos, ¡tendrás la mejor solución para el rompecabezas gigante! 👍
   Por ejemplo, si quieres encontrar el camino más corto de tu casa a la escuela 🏠➡️🏫, y ese camino pasa por el parque 🌳, entonces el pedacito de "casa al parque" y el pedacito de "parque a la escuela" ¡también deben ser los más cortos posibles!

**La Magia de Recordar** 💡

Si nuestro problema tiene estas dos "superpoderes", la Programación Dinámica nos permite ser súper eficientes. Al "recordar" (técnicamente se llama memoización o tabulación) las soluciones a los subproblemas que se repiten, ¡evitamos hacer el mismo trabajo una y otra vez! Esto puede convertir un problema que tardaría años en resolverse en uno que se soluciona en segundos. ¡Increíble! 🚀

### Memoización: El Arte de Recordar

Una técnica fundamental en Programación Dinámica es la **memoización**. La palabra "memoización" proviene del latín "memorandum", que significa "cosa que se debe recordar". En el contexto de algoritmos, la memoización es una técnica de optimización utilizada para acelerar programas de computadora al almacenar los resultados de llamadas a funciones costosas y devolver el resultado almacenado cuando las mismas entradas ocurren nuevamente.

Consideremos el cálculo del $n$-ésimo número de Fibonacci, donde $F_0 = 0$, $F_1 = 1$, y $F_n = F_{n-1} + F_{n-2}$ para $n > 1$. Una implementación recursiva directa es simple:

In [ ]:
def fibonacci_recursivo(n):
    if n <= 1:
        return n
    return fibonacci_recursivo(n - 1) + fibonacci_recursivo(n - 2)

In [ ]:
%%timeit
fibonacci_recursivo(32)

### Fibonacci: El Ejemplo Clásico 🐇

La versión recursiva ingenua de Fibonacci recalcula los mismos valores una y otra vez ($O(2^n)$).
`fib(5)` llama a `fib(4)` y `fib(3)`. `fib(4)` llama a `fib(3)` y `fib(2)`. ¡`fib(3)` se calcula dos veces!

**Solución con Memoización (Top-Down)**:
Guardamos los resultados en un diccionario (`cache`). Antes de calcular, preguntamos: "¿Ya me sé la respuesta?".
Esto reduce la complejidad a $O(n)$.

En Python, podemos usar el decorador `@functools.cache` para hacer esto automáticamente. ✨

In [ ]:
def fibonacci_memoizado(n, cache={}):
    # Verificamos si el resultado ya está en el cache (diccionario)
    if n in cache:
        return cache[n]

    # Casos base
    if n <= 1:
        resultado = n
    else:
        # Calculamos el resultado recursivamente y lo almacenamos en el cache
        resultado = fibonacci_memoizado(n - 1, cache) + fibonacci_memoizado(
            n - 2, cache
        )

    cache[n] = resultado
    return resultado

In [ ]:
%%timeit
fibonacci_memoizado(32)

In [ ]:
import functools


@functools.cache
def fibonacci_cache(n):
    if n <= 1:
        return n
    return fibonacci_cache(n - 1) + fibonacci_cache(n - 2)

In [ ]:
%%timeit
fibonacci_cache(32)

### La Receta de la Programación Dinámica 📜

Para aplicar PD, tu problema debe cumplir dos propiedades:

1.  **Subestructura Óptima**: La solución óptima global se construye a partir de soluciones óptimas de subproblemas. (Ej: El camino más corto A->C pasa por B; entonces A->B también debe ser el camino más corto).
2.  **Subproblemas Superpuestos**: Los mismos subproblemas se resuelven una y otra vez. (Ej: Fibonacci(5) pide Fibonacci(3) varias veces).

**Pasos:**
1.  **Identificar** la estructura recursiva.
2.  **Definir** la relación de recurrencia (fórmula matemática).
3.  **Calcular** las soluciones:
    *   *Top-Down* (Memoización): Recursión + Cache.
    *   *Bottom-Up* (Tabulación): Llenar una tabla iterativamente desde los casos base.
4.  **(Opcional) Reconstruir** la solución siguiendo el rastro en la tabla.

### PD Aplicada: Mochila 0/1 (Knapsack) 🎒

Usaremos el enfoque **Bottom-Up** (Tabulación).
Construiremos una tabla $K[i][w]$ donde:
*   $i$: Consideramos solo los primeros $i$ objetos.
*   $w$: Capacidad actual de la mochila.
*   $K[i][w]$: Valor máximo posible.

**La Decisión (Recurrencia):**
Para el objeto $i$ con peso $w_i$ y valor $v_i$:
1.  **No incluirlo**: El valor es el mismo que con $i-1$ objetos ($K[i-1][w]$).
2.  **Incluirlo** (si cabe): Sumamos su valor al mejor valor posible con el espacio restante ($v_i + K[i-1][w-w_i]$).

$$K[i][w] = \max(K[i-1][w], \quad v_i + K[i-1][w-w_i])$$

Llenamos la tabla fila por fila. La respuesta final estará en $K[n][W]$.

In [ ]:
def knapsack_dp(W, w, v):
   n = len(w)

   # Tablas para DP
   K = [[0 for _ in range(W + 1)] for _ in range(n + 1)]  # Tabla de valores máximos
   P = [[0 for _ in range(W + 1)] for _ in range(n + 1)]  # Tabla de decisiones

   # Llenar tablas K y P
   for i in range(n + 1):
      for j in range(W + 1):
         if i > 0 and j > 0:
            wi = w[i - 1]  # Peso del ítem actual
            vi = v[i - 1]  # Valor del ítem actual

            excl = K[i - 1][j]  # Caso: excluir el ítem actual
            incl = -1
            if wi <= j:
               incl = vi + K[i - 1][j - wi]  # Caso: incluir el ítem actual

            if incl > excl:
               K[i][j] = incl  # Guardar el valor máximo al incluir
               P[i][j] = 1  # Marcar que el ítem fue incluido
            else:
               K[i][j] = excl  # Guardar el valor máximo al excluir
               P[i][j] = 0  # Marcar que el ítem no fue incluido

   # Reconstruir solución
   sol = []
   i, j = n, W
   while i > 0 and j > 0:
      if P[i][j] == 1:  # Si el ítem fue incluido
         sol.append(i - 1)  # Agregar el índice del ítem a la solución
         j -= w[i - 1]  # Reducir la capacidad restante
      i -= 1  # Pasar al ítem anterior

   sol.reverse()  # Invertir para obtener el orden original
   return K[n][W], sol  # Retornar el valor máximo y los ítems seleccionados

In [ ]:
# Ejemplo de uso
W = 10
w = [2, 3, 4, 5]
v = [3, 7, 2, 9]

max_val, sel_items = knapsack_dp(W, w, v)

print(f"Valor máximo: {max_val}")  # Imprimir el valor máximo obtenido
print(f"Ítems seleccionados: {sel_items}")  # Imprimir los índices de los ítems seleccionados
print("Pesos seleccionados:", [w[i] for i in sel_items])
print("Valores seleccionados:", [v[i] for i in sel_items])


### Análisis y Reconstrucción 🕵️‍♀️

*   **Complejidad**: $O(n \cdot W)$ tanto en tiempo como en espacio.
    *   Nota: Esto es *pseudo-polinomial* (depende de la magnitud de $W$, no solo de $n$).
*   **Reconstrucción**: Para saber *qué* objetos llevamos, usamos una tabla auxiliar $P$ (Path) o simplemente retrocedemos en $K$:
    *   Si $K[i][w] \neq K[i-1][w]$, significa que incluimos el objeto $i$. Restamos su peso y seguimos buscando.

¡Veamos el código completo con reconstrucción! 👇

### Ejercicios 🧠

1.  **Visualización**: Modifica el código para imprimir la tabla $K$ de forma bonita (usando `pandas` o `tabulate`) y observa cómo se propagan los valores.
2.  **Optimización de Espacio**: ¿Realmente necesitamos toda la tabla $K$? Observa que para calcular la fila $i$, solo necesitas la fila $i-1$. Intenta implementar una versión que use solo dos filas (o incluso una sola) para ahorrar memoria ($O(W)$ espacio). *Nota: Reconstruir la solución se vuelve más difícil.*
3.  **Otros Problemas Clásicos**: Investiga y resuelve:
    *   **Cambio de Monedas**: Mínimo número de monedas para dar un cambio $C$.
    *   **LCS (Longest Common Subsequence)**: La subsecuencia común más larga entre dos textos.

### Discusión con tu IA 🤖

1.  **Memoización vs Tabulación**: Pregunta cuándo es mejor usar uno u otro. (Pista: ¿Qué pasa si no necesitas llenar toda la tabla?).
2.  **Complejidad Pseudo-polinomial**: Pide que te explique por qué $O(n \cdot W)$ no es técnicamente polinomial. ¿Qué pasa si $W$ es $10^{18}$?